In [1]:
import nltk

In [2]:
lines = [line.rstrip('\n') for line in open('data-train.txt')]

# Подготовка данных
Сгруппируем все предложения для конкретного языка

In [3]:
def text_filter(text):
    return [c.lower() for c in text if c.isalpha() or c == ' ']

l = {}
l_cnt = {}

for i in range(1, len(lines)):
    lang, text = lines[i].split('\t', 1)
    
    f = l_cnt.get(lang, 0)
    l_cnt[lang] = f + 1
    
    if (l.get(lang) is None):
        l[lang] = [text]
    else:
        l[lang].append(text) 


In [4]:
from nltk.probability import FreqDist
from nltk import trigrams

words_fd = {}

for lang, texts in l.items():
    words_fd[lang] = FreqDist()
    for text in texts:
        symbols = []
        for c in text:
            symbols.append(c)
#             symbols.append(c.lower())
#             if c.isalpha() or c == ' ':
#                 symbols.append(c.lower())
        words_fd[lang].update(trigrams(symbols))

# Модель 1
Для каждого языка построим триграмную языковую модель со вглаживанием KneserNey.

In [5]:
from nltk.probability import KneserNeyProbDist

kn = {}
for lang in l.keys():
    kn[lang] = KneserNeyProbDist(words_fd[lang])


In [6]:
l_arr = ['be','bg','cs','de','el','en','es','fr','hr','hsb','hy','it','la','lt','lv','mk','nl','pl','pt','ro','ru','sk','sl','sr','sv','uk']

l_ids = {}
l_i = 0
l_from_id = {}

for lang in l_arr:
    l_ids[lang] = l_i
    l_from_id[l_i] = lang
    l_i += 1

## Классификация
Для классификации языка текста, воспользуемся формулой Байеса:
$$ P(l \mid t) = \frac{P(t \mid l) P(l)}{P(t)} \propto P(t \mid l) P(l), $$
где $ l $ - язык, $ t $ - текст, $ P(l) $ - априорная вероятность языка $ l $ и $ P(t\mid l) $ - вероятность текста  в рамках языковой модели. 

Тогда получаем, что $$ l' = \arg\max\limits_{l} \left\{ softmax \left( P(t \mid l_1) P(l_1), \ldots, P(t \mid l_n) P(l_n) \right) \right\} . $$
Заметим, что разные языки встречаются в обучающей выборке с разной частотой, поэтому $ P(l) $ несет в себе важную информацию. 

In [7]:
def softmax(w, t=1.0):
    m = np.max(w)
    e = np.exp((np.array(w) - m) / t)
    dist = e / np.sum(e)
    return dist

In [8]:
import math
import numpy as np

def lang_prob(lang):
    return l_cnt[lang] / (len(lines) - 1)

def langs_prob(text):
    probs = [0] * len(l.keys())
    
    for lang in l.keys():
        prob = 0.0
        for i in range(2, len(text)):
            prob += math.log(kn[lang].prob((text[i-2], text[i-1], text[i])) + 0.1, 2)
        prob += math.log(lang_prob(lang), 2)
        probs[l_ids[lang]] = prob
        
    return softmax(probs)

def predict_lang(text):
    return l_from_id[np.argmax(langs_prob(text))]

In [10]:
lines = [line.rstrip('\n') for line in open('data-test.txt')]

## Результаты
При отправке на kaggle: <b>0.02055</b>
### Модификации
Были предприняты различные попытки по "чистке" входных данных:
1. Перевод всех символов в нижний регистр
2. Удаление всех не alphanum символов (При отправке на kaggle: <b>0.02314</b>)
3. Модификации 1 и 2 вместе (При отправке на kaggle: <b>0.02832</b>)

К сожалению, это все приводило только к ухудшению результата.

In [11]:
import csv


test_out = open('test.out', 'w')
writer = csv.writer(test_out)
writer.writerow(['id', 'be','bg','cs','de','el','en','es','fr','hr','hsb','hy','it','la','lt','lv','mk','nl','pl','pt','ro','ru','sk','sl','sr','sv','uk'])

l_count = len(l.keys())

for i in range(1, len(lines)):
    k, text = lines[i].split('\t', 1)

#     text = text_filter(text)

    probs = [0] * l_count
    probs[l_ids[predict_lang(text)]] = 1
    probs = [k] + probs
    writer.writerow(probs)
        
test_out.flush()
test_out.close()